# 식당 카테고리 재배치 프로세스

## 개요
식당 데이터의 카테고리를 체계적으로 재배치하는 과정입니다. 카테고리는 3단계로 구성되어 있습니다:
- diner_category_middle (대분류)
- diner_category_small (중분류) 
- diner_category_detail (소분류)

### 2. 카테고리 하향 조정
특정 middle 카테고리를 small로 내리는 작업 수행:
- 예시: "뷔페/패밀리레스토랑" -> [패밀리레스토랑, 뷔페, 푸드코트]
- small 값을 detail로 이동
- middle 값을 small로 이동
- 새로운 middle 카테고리 할당

### 3. 부분 카테고리 하향 조정
특정 조건의 카테고리만 선별적으로 하향 조정:
- 예시: "분식" 중 "떡볶이"만 "한식"으로 조정
- 조건부 필터링을 통해 선택적 카테고리 이동

### 4. 동일 레벨 카테고리 재분류 
같은 단계 내에서 카테고리 재분류:
- 예시: "치킨" -> [프라이드치킨, 구운치킨]
- 기준에 따라 같은 레벨의 카테고리를 새롭게 분류

### 5. 카테고리 상향 및 재배치
카테고리를 상위로 올리고 재배치:
- small을 middle로 상향
- detail을 small로 상향
- 카테고리 간 일관성 유지

### 6. 이중 하향 조정
두 단계 아래로 내리는 조정:
- 특정 브랜드/매장을 detail로 하향
- 적절한 small/middle 카테고리 할당

## 주의사항
- 카테고리 이동 전 기존 매핑 검증 필요
- 일관된 카테고리 체계 유지
- 예외 케이스에 대한 별도 처리 고려



In [52]:
import pandas as pd

diner_df = pd.read_csv("../data/diner/diner_df_20241211_yamyam.csv")
before_diner_df = pd.read_csv("../data/diner/diner_df_20241204_yamyam.csv")

In [93]:
# diner_category_middle과 diner_category_small의 유니크한 값 추출
unique_middle_categories = diner_df["diner_category_middle"].unique()
unique_small_categories = diner_df["diner_category_small"].unique()

### 1. 카테고리 매핑 사전 생성
- 기존 카테고리 구조를 딕셔너리 형태로 저장
- groupby를 통해 middle-small 카테고리 간 관계 매핑

In [151]:
# 그룹화하여 딕셔너리 생성
category_dict = (
    diner_df.groupby("diner_category_middle")["diner_category_small"]
    .apply(lambda x: set(x.dropna()))  # NaN 제거
    .to_dict()
)
category_dict

{'간식': {'떡,한과', '아이스크림', '외국 간식', '제과,베이커리', '철판요리', '초콜릿, 사탕', '토스트'},
 '뷔페/패밀리레스토랑': {'뷔페', '패밀리레스토랑', '푸드코트'},
 '술집': {'실내포장마차', '오뎅바', '와인바', '일본식주점', '칵테일바', '호프,요리주점'},
 '아시아음식': {'동남아음식', '인도음식', '튀르키예음식'},
 '양식': {'멕시칸,브라질',
  '샌드위치',
  '샐러드',
  '스테이크,립',
  '스페인음식',
  '이탈리안',
  '패스트푸드',
  '프랑스음식',
  '피자',
  '해산물',
  '햄버거'},
 '육류,고기': {'갈비', '곱창,막창', '닭요리', '돼지고기', '불고기,두루치기', '소고기', '오리', '족발,보쌈'},
 '일식': {'돈까스,우동', '일본식라면', '일식집', '참치회', '철판요리', '초밥,롤'},
 '중식': {'양꼬치', '중국요리'},
 '치킨': {'구운치킨', '닭강정', '프라이드치킨'},
 '퓨전요리': {'퓨전일식', '퓨전중식', '퓨전한식'},
 '한식': {'감자탕',
  '곰탕',
  '구내식당',
  '국밥',
  '국수',
  '기사식당',
  '냉면',
  '도시락,덮밥',
  '두부전문점',
  '떡볶이',
  '분식',
  '사철탕,영양탕',
  '샤브샤브',
  '설렁탕',
  '수제비',
  '순대',
  '쌈밥',
  '주먹밥',
  '죽',
  '찌개,전골',
  '한정식',
  '해장국'},
 '해물,생선': {'게,대게', '굴,전복', '매운탕,해물탕', '복어', '장어', '조개', '찜,조림,볶음', '추어', '회'}}

### 2. 카테고리 하향 조정
특정 middle 카테고리를 small로 내리는 작업 수행:
- 예시: "뷔페/패밀리레스토랑" -> [패밀리레스토랑, 뷔페, 푸드코트]
- small 값을 detail로 이동
- middle 값을 small로 이동
- 새로운 middle 카테고리 할당

In [21]:
lowering_middle_categories = {
    "뷔페/패밀리레스토랑": ["패밀리레스토랑", "뷔페", "푸드코트"],
    "일식": ["철판요리"],
}

for after_category, befor_categorries in lowering_middle_categories.items():
    # 0. 조정할 row 설정
    target_rows = diner_df["diner_category_middle"].isin(befor_categorries)

    # 1. 원래 diner_category_small 값을 diner_category_detail로 이동
    diner_df.loc[target_rows, "diner_category_detail"] = diner_df.loc[
        target_rows, "diner_category_small"
    ]

    # 2. diner_category_small 값을 각각 befor_categorys로 이동
    diner_df.loc[target_rows, "diner_category_small"] = diner_df.loc[
        target_rows, "diner_category_middle"
    ]

    # 3. diner_category_middle 값을 after_category으로 수정
    diner_df.loc[target_rows, "diner_category_middle"] = after_category

### 3. 부분 카테고리 하향 조정
특정 조건의 카테고리만 선별적으로 하향 조정:
- 예시: "분식" 중 "떡볶이"만 "한식"으로 조정
- 조건부 필터링을 통해 선택적 카테고리 이동

In [22]:
partly_lowering_middle_categories = {
    "한식": {
        "lowering_middle_categories": [
            "분식",
            "샤브샤브",
            "기사식당",
            "구내식당",
            "도시락",
        ],
        "partly_lowering_middle_category": "분식",
        "partly_lowering_small_categories": ["떡볶이"],
    },
    "양식": {
        "lowering_middle_categories": ["패스트푸드", "샐러드"],
        "partly_lowering_middle_category": "패스트푸드",
        "partly_lowering_small_categories": ["샌드위치"],
    },
}

for after_category, category_info in partly_lowering_middle_categories.items():
    befor_categorries = category_info["lowering_middle_categories"]
    part_middle_category = category_info["partly_lowering_middle_category"]
    part_small_categories = category_info["partly_lowering_small_categories"]

    # 1. diner_category_small이 part_small_categories인 경우 diner_category_middle을 after_category로 변경
    update_condition = (diner_df["diner_category_middle"] == part_middle_category) & (
        diner_df["diner_category_small"].isin(part_small_categories)
    )

    diner_df.loc[update_condition, "diner_category_middle"] = after_category

    # 조건 필터링
    target_rows = diner_df["diner_category_middle"].isin(befor_categorries)

    diner_df.loc[target_rows, "diner_category_detail"] = diner_df.loc[
        target_rows, "diner_category_small"
    ]

    diner_df.loc[target_rows, "diner_category_small"] = diner_df.loc[
        target_rows, "diner_category_middle"
    ]

    # 3. diner_category_middle 값을 after_category으로 수정
    diner_df.loc[target_rows, "diner_category_middle"] = after_category

In [23]:
diner_df["diner_category_middle"].value_counts()

diner_category_middle
한식            20591
육류,고기         11838
술집             9608
양식             8871
간식             7448
일식             6268
해물,생선          5687
치킨             5491
중식             5256
아시아음식          1670
퓨전요리            774
뷔페/패밀리레스토랑      738
야식               19
Name: count, dtype: int64

### 4. 동일 레벨 카테고리 재분류 
같은 단계 내에서 카테고리 재분류:
- 예시: "치킨" -> [프라이드치킨, 구운치킨]
- 기준에 따라 같은 레벨의 카테고리를 새롭게 분류

In [147]:
diner_df[diner_df["diner_category_middle"] == "치킨"][
    "diner_category_small"
].value_counts()

diner_category_small
프라이드치킨    4589
구운치킨       517
닭강정        380
Name: count, dtype: int64

In [17]:
chiken_category = {
    "치킨/구이": [
        "지코바",
        "계림원",
        "훌랄라",
        "굽네치킨",
        "굽자나",
        "기영이숯불두마리치킨",
        "꾸브라꼬숯불두마리치킨",
        "누구나홀딱반한닭",
        "동근이숯불두마리치킨",
        "땅땅치킨",
        "불로만숯불바베큐",
        "오븐마루치킨",
        "오븐에빠진닭",
        "코리안바베큐",
        "화락바베큐치킨",
        "바비큐보스",
        "이춘봉인생치킨",
        "구운치킨",
    ]
}

target_categories = ["치킨"]
target_rows = (diner_df["diner_category_middle"].isin(target_categories)) & (
    ~diner_df["diner_category_small"].isin(["프라이드치킨", "구운치킨", "닭강정"])
)

diner_df.loc[target_rows, "diner_category_detail"] = diner_df.loc[
    target_rows, "diner_category_small"
]

# 1. diner_category_small이 "치킨/구이" 리스트에 있는 경우 "구운치킨"으로 변경
grilled_chicken = chiken_category["치킨/구이"]
diner_df.loc[
    target_rows & diner_df["diner_category_small"].isin(grilled_chicken),
    "diner_category_small",
] = "구운치킨"

# 2. diner_category_small이 "치킨/구이" 리스트에 없는 경우 "프라이드치킨"으로 변경
diner_df.loc[
    target_rows & ~diner_df["diner_category_small"].isin(grilled_chicken),
    "diner_category_small",
] = "프라이드치킨"

In [24]:
diner_df[diner_df["diner_category_small"].isin(["구운치킨"])][
    "diner_category_detail"
].value_counts()

diner_category_detail
굽네치킨           178
지코바             92
누구나홀딱반한닭        37
기영이숯불두마리치킨      26
계림원             25
오븐마루치킨          25
훌랄라             25
동근이숯불두마리치킨      23
화락바베큐치킨         23
꾸브라꼬숯불두마리치킨     17
오븐에빠진닭          17
땅땅치킨            10
코리안바베큐           8
이춘봉인생치킨          5
바비큐보스            3
불로만숯불바베큐         2
굽자나              1
Name: count, dtype: int64

### 5. 카테고리 상향 및 재배치
카테고리를 상위로 올리고 재배치:

In [25]:
upper_replacing_small_categories = {
    "육류,고기": {
        "replacing_categories": {
            "닭요리": [],
            "갈비": ["닥엔돈스"],
            "족발,보쌈": [],
            "곱창,막창": [],
            "돼지고기": [
                "흑다돈",
                "정가대박집",
                "서래갈매기",
                "쟁반집8292",
                "신마포갈매기",
                "고기싸롱",
                "엉터리생고기",
                "목구멍",
                "식껍",
                "고기극찬",
                "원조부안집",
                "고반식당",
                "맛찬들왕소금구이",
                "삼산회관",
                "고돼지",
                "미진축산",
                "삼겹살",
            ],
            "소고기": [
                "보리네생고깃간",
                "착한고기",
                "이차돌",
                "한마음정육식당",
                "논골집",
                "돌배기집",
                "차돌풍",
                "육칠이",
                "본가",
                "목우촌웰빙마을",
                "육회지존",
                "무쏘",
                "그램그램",
                "김형제고기의철학",
            ],
            "오리": [],
            "불고기,두루치기": ["새마을식당"],
        },
    },
    "해물,생선": {
        "replacing_categories": {
            "회": ["동원참치", "오징어나라"],
            "장어": [],
            "추어": [],
            "찜,조림,볶음": [
                "아구",
                "김영희강남동태찜",
                "어부네코다리조림",
                "황금코다리",
                "김명자낙지마당",
                "마린보이코다리1번가",
            ],
            "게,대게": [],
            "조개": [],
            "매운탕,해물탕": ["바다양푼이동태탕", "연안식당"],
            "복어": [],
            "굴,전복": [],
        }
    },
}

for target_category, category_info in upper_replacing_small_categories.items():
    replacing_categories = category_info["replacing_categories"]

    # 1. diner_category_detail 값이 meat_category의 value에 있으면 key값으로 변경
    for key, values in replacing_categories.items():
        diner_df.loc[
            diner_df["diner_category_detail"].isin(values), "diner_category_detail"
        ] = key

    # 2. diner_category_small이 '육류,고기'인 경우 값 올리기
    target_rows = diner_df["diner_category_small"].isin([target_category])

    # diner_category_middle로 값 올리기
    diner_df.loc[target_rows, "diner_category_middle"] = diner_df.loc[
        target_rows, "diner_category_small"
    ]

    # diner_category_small에 diner_category_detail 값 올리기
    diner_df.loc[target_rows, "diner_category_small"] = diner_df.loc[
        target_rows, "diner_category_detail"
    ]

### 6. 이중 하향 조정
두 단계 아래로 내리는 조정:
- 특정 브랜드/매장을 detail로 하향
- 적절한 small/middle 카테고리 할당

In [26]:
down_category = {
    "분식": [],
    "국수": [],
    "떡볶이": [],
    "순대": [],
    "국밥": [],
    "찌개,전골": ["한식밥상", "혼밥대왕"],
    "샤브샤브": [],
    "감자탕": [],
    "냉면": [],
    "해장국": [],
    "도시락, 덮밥": ["뜸들이다", "도시락", "핵밥", "순수덮밥"],
    "한정식": [],
    "두부전문점": [],
    "곰탕": [],
    "죽": [],
    "설렁탕": [],
    "기사식당": [],
    "쌈밥": [],
    "구내식당": [],
    "사철탕,영양탕": [],
    "주먹밥": [],
    "수제비": [],
    "찜, 조림, 볶음": ["오봉집", "해탄"],
    "곱창,막창": ["승도리네 곱도리탕"],
    "불고기,두루치기": ["제육의법칙"],
    "퓨전일식": ["아비꼬", "코코이찌방야"],
    "퓨전중식": ["니뽕내뽕"],
    "퓨전한식": ["돼지게티"],
    "돈까스,우동": ["백소정", "토끼정"],
    "철판요리": ["타코비", "호시타코야끼"],
    "초밥,롤": ["오니기리"],
    "일본식라면": ["면식당"],
    "초콜릿, 사탕": ["초콜릿", "왕코미네탕후루", "황제탕후루"],
    "제과,베이커리": ["도넛"],
    "외국 간식": ["비첸향"],
    "중국요리": ["탄탄면공방"],
    "이탈리안": ["서가앤쿡", "파스토보이"],
}

for k, v in down_category.items():
    target_categories = v
    target_rows = diner_df["diner_category_small"].isin(target_categories)

    diner_df.loc[target_rows, "diner_category_detail"] = diner_df.loc[
        target_rows, "diner_category_small"
    ]

    # 3. diner_category_middle 값을 "뷔페/패스트푸드"으로 수정
    diner_df.loc[target_rows, "diner_category_small"] = k

    # middle 카테고리 재배치
    for c_middle, c_small_list in category_dict.items():
        if k in c_small_list:
            diner_df.loc[target_rows, "diner_category_middle"] = c_middle

### middle만 바꾸기

In [22]:
target_rows = diner_df["diner_category_small"].isin(["닭강정"])

diner_df.loc[target_rows, "diner_category_middle"] = "치킨"

In [152]:
diner_df["diner_category_small"].value_counts().to_dict()

{'제과,베이커리': 4976,
 '프라이드치킨': 4589,
 '분식': 4584,
 '호프,요리주점': 4314,
 '중국요리': 3636,
 '국수': 2166,
 '돈까스,우동': 2066,
 '피자': 2061,
 '닭요리': 1951,
 '회': 1862,
 '일본식주점': 1722,
 '떡볶이': 1526,
 '실내포장마차': 1458,
 '갈비': 1437,
 '동남아음식': 1315,
 '곱창,막창': 1305,
 '족발,보쌈': 1273,
 '초밥,롤': 1269,
 '순대': 1233,
 '이탈리안': 1147,
 '돼지고기': 1125,
 '패스트푸드': 1045,
 '국밥': 974,
 '찌개,전골': 974,
 '떡,한과': 851,
 '아이스크림': 826,
 '샐러드': 800,
 '햄버거': 729,
 '샌드위치': 718,
 '샤브샤브': 692,
 '양꼬치': 678,
 '와인바': 655,
 '감자탕': 651,
 '냉면': 638,
 '뷔페': 560,
 '구운치킨': 517,
 '도시락,덮밥': 511,
 '해장국': 490,
 '일식집': 476,
 '참치회': 463,
 '장어': 434,
 '칵테일바': 405,
 '일본식라면': 402,
 '추어': 396,
 '한정식': 385,
 '닭강정': 380,
 '찜,조림,볶음': 379,
 '퓨전한식': 341,
 '두부전문점': 318,
 '오뎅바': 298,
 '곰탕': 296,
 '멕시칸,브라질': 293,
 '토스트': 285,
 '죽': 281,
 '오리': 277,
 '설렁탕': 255,
 '게,대게': 210,
 '퓨전일식': 193,
 '매운탕,해물탕': 184,
 '조개': 183,
 '스테이크,립': 181,
 '인도음식': 161,
 '불고기,두루치기': 143,
 '복어': 135,
 '기사식당': 117,
 '쌈밥': 112,
 '패밀리레스토랑': 109,
 '구내식당': 107,
 '사철탕,영양탕': 91,
 '굴,전복': 83,
 '철판요리'

## 주의사항
- 카테고리 이동 전 기존 매핑 검증 필요
- 일관된 카테고리 체계 유지
- 예외 케이스에 대한 별도 처리 고려

In [153]:
# TODO: small 카테고리
unprocessed_small_category = []

In [154]:
diner_df[diner_df["diner_category_small"].isin(unprocessed_small_category)][
    "diner_category_middle"
].value_counts()

Series([], Name: count, dtype: int64)

In [38]:
diner_df.to_csv(
    "/home/elinha/GNN/data/diner/diner_df_20241211_yamyam.csv",
    index=False,
    encoding="utf-8-sig",
)

diner_category_small - null 값처리

In [79]:
# 2. diner_category_small이 '육류,고기'인 경우 값 올리기
# 조건: 'diner_category_middle'가 '야식'이고 'diner_name'에 '모퉁이'가 포함된 경우
target_rows = (diner_df["diner_category_middle"] == "야식") & diner_df[
    "diner_name"
].str.contains("모퉁이", na=False)
target_rows.value_counts()

False    84283
True         1
Name: count, dtype: int64

In [140]:
# 조건: 'diner_category_middle'가 '야식'이고 'diner_name'에 '모퉁이'가 포함된 경우
target_rows = (diner_df["diner_category_middle"].isna()) & diner_df[
    "diner_name"
].str.contains("뷔페", na=False)
print(target_rows.value_counts())

# 조건에 해당하는 'diner_category_small'과 'diner_category_middle' 업데이트
diner_df.loc[target_rows, "diner_category_small"] = "뷔페"
diner_df.loc[target_rows, "diner_category_middle"] = "뷔페/패밀리레스토랑"

False    84283
True         1
Name: count, dtype: int64


In [139]:
print(category_dict["뷔페/패밀리레스토랑"])

{'뷔페', '패밀리레스토랑', '푸드코트'}


## 정리전 후 시각화

In [145]:
import pandas as pd
import plotly.express as px


# middle별 음식점 수 계산
middle_counts = diner_df["diner_category_middle"].value_counts().reset_index()
middle_counts.columns = ["Category", "Count"]

# small별 음식점 수 계산
small_counts = diner_df["diner_category_small"].value_counts().reset_index()
small_counts.columns = ["Category", "Count"]

# middle 시각화
middle_fig = px.bar(
    middle_counts,
    x="Category",
    y="Count",
    title="Number of Restaurants by Middle Category",
    labels={"Category": "Middle Category", "Count": "Number of Restaurants"},
)

# small 시각화
small_fig = px.bar(
    small_counts,
    x="Category",
    y="Count",
    title="Number of Restaurants by Small Category",
    labels={"Category": "Small Category", "Count": "Number of Restaurants"},
)

# 결과 시각화
middle_fig.show()
small_fig.show()

In [3]:
import pandas as pd
import plotly.express as px


# middle별 음식점 수 계산
middle_counts = before_diner_df["diner_category_middle"].value_counts().reset_index()
middle_counts.columns = ["Category", "Count"]

# small별 음식점 수 계산
small_counts = before_diner_df["diner_category_small"].value_counts().reset_index()
small_counts.columns = ["Category", "Count"]

# middle 시각화
middle_fig = px.bar(
    middle_counts,
    x="Category",
    y="Count",
    title="Number of Restaurants by Middle Category",
    labels={"Category": "Middle Category", "Count": "Number of Restaurants"},
)

# small 시각화
small_fig = px.bar(
    small_counts,
    x="Category",
    y="Count",
    title="Number of Restaurants by Small Category",
    labels={"Category": "Small Category", "Count": "Number of Restaurants"},
)

# 결과 시각화
middle_fig.show()
small_fig.show()

In [146]:
diner_df.to_csv(
    "/home/elinha/GNN/data/diner/diner_df_20241211_yamyam.csv",
    index=False,
    encoding="utf-8-sig",
)